In [1]:
#pip install pydub
#apt-get install ffmpeg libavcodec-extra
#pip install ffprobe-python
#pip install tensorflow_datasets -U
#pip install -q tensorflow-io

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import sys
sys.path.append('/usr/local/lib/python3.6/dist-packages')
print(tf.__version__)
import numpy as np
import wave
import os
#import tensorflow_io as tfio
from tensorflow.python.platform import gfile

2.1.0


In [3]:
sys.path.append('/usr/local/lib/python3.6/dist-packages')
import ffprobe
download=True
name="speech_commands"
ds = tfds.load(name,download=download,shuffle_files=True)
num_parallel_calls = 4
prefetch_size = 2
resize_shape = False

dct_coefficient_count=10 
window_size_ms=40 
window_stride_ms=20
clip_duration_ms=1000 #expected duration in ms
sample_rate=16000
label_count=12
background_frequency = 0.8
background_volume_range_=0.1

def prepare_model_settings(label_count, sample_rate, clip_duration_ms,
                               window_size_ms, window_stride_ms,
                               dct_coefficient_count,background_frequency):
      """Calculates common settings needed for all models.
      Args:
        label_count: How many classes are to be recognized.
        sample_rate: Number of audio samples per second.
        clip_duration_ms: Length of each audio clip to be analyzed.
        window_size_ms: Duration of frequency analysis window.
        window_stride_ms: How far to move in time between frequency windows.
        dct_coefficient_count: Number of frequency bins to use for analysis.
      Returns:
        Dictionary containing common settings.
      """
      desired_samples = int(sample_rate * clip_duration_ms / 1000)
      window_size_samples = int(sample_rate * window_size_ms / 1000)
      window_stride_samples = int(sample_rate * window_stride_ms / 1000)
      length_minus_window = (desired_samples - window_size_samples)
      if length_minus_window < 0:
        spectrogram_length = 0
      else:
        spectrogram_length = 1 + int(length_minus_window / window_stride_samples)
      fingerprint_size = dct_coefficient_count * spectrogram_length
      return {
          'desired_samples': desired_samples,
          'window_size_samples': window_size_samples,
          'window_stride_samples': window_stride_samples,
          'spectrogram_length': spectrogram_length,
          'dct_coefficient_count': dct_coefficient_count,
          'fingerprint_size': fingerprint_size,
          'label_count': label_count,
          'sample_rate': sample_rate,
          'background_frequency': 0.8,
          'background_volume_range_': 0.1
      }
    
def get_preprocess_audio_func(model_settings,is_training=False,background_data = []):
    def prepare_processing_graph(next_element):
        """Builds a TensorFlow graph to apply the input distortions.
        Creates a graph that loads a WAVE file, decodes it, scales the volume,
        shifts it in time, adds in background noise, calculates a spectrogram, and
        then builds an MFCC fingerprint from that.
        This must be called with an active TensorFlow session running, and it
        creates multiple placeholder inputs, and one output:
          - wav_filename_placeholder_: Filename of the WAV to load.
          - foreground_volume_placeholder_: How loud the main clip should be.
          - time_shift_padding_placeholder_: Where to pad the clip.
          - time_shift_offset_placeholder_: How much to move the clip in time.
          - background_data_placeholder_: PCM sample data for background noise.
          - background_volume_placeholder_: Loudness of mixed-in background.
          - mfcc_: Output 2D fingerprint of processed audio.
        Args:
          model_settings: Information about the current model being trained.
        """
        desired_samples = model_settings['desired_samples']
        background_frequency = model_settings['background_frequency']
        background_volume_range_= model_settings['background_volume_range_']
        wav_decoder = tf.cast(next_element['audio'], tf.float32)
        wav_decoder = wav_decoder/tf.reduce_max(wav_decoder)
        wav_decoder = tf.pad(wav_decoder,[[0,desired_samples-tf.shape(wav_decoder)[-1]]]) #Previously, decode_wav was used with desired_samples as the length of array. The default option of this function was to pad zeros if the desired samples are not found
        # Allow the audio sample's volume to be adjusted.
        foreground_volume_placeholder_ = tf.constant(1,dtype=tf.float32)
        
        scaled_foreground = tf.multiply(wav_decoder,
                                        foreground_volume_placeholder_)
        # Shift the sample's start position, and pad any gaps with zeros.
        time_shift_padding_placeholder_ = tf.constant([[2,2]], tf.int32)
        time_shift_offset_placeholder_ = tf.constant([2],tf.int32)
        scaled_foreground.shape
        padded_foreground = tf.pad(scaled_foreground, time_shift_padding_placeholder_, mode='CONSTANT')
        sliced_foreground = tf.slice(padded_foreground, time_shift_offset_placeholder_, [desired_samples])
        
    
        if is_training and background_data != []:
            background_volume_range = tf.constant(background_volume_range_,dtype=tf.float32)
            background_index = np.random.randint(len(background_data))
            background_samples = background_data[background_index]
            background_offset = np.random.randint(0, len(background_samples) - desired_samples)
            background_clipped = background_samples[background_offset:(background_offset + desired_samples)]
            background_clipped = tf.squeeze(background_clipped)
            #print(background_clipped)
            #background_reshaped = background_clipped.reshape([desired_samples, 1])
            background_reshaped = tf.pad(background_clipped,[[0,desired_samples-tf.shape(wav_decoder)[-1]]])
            background_reshaped = tf.cast(background_reshaped, tf.float32)
            if np.random.uniform(0, 1) < background_frequency:
                background_volume = np.random.uniform(0, background_volume_range_)
            else:
                background_volume = 0
            #background_data_placeholder_ = tf.placeholder(tf.float32, [desired_samples, 1])
            background_volume_placeholder_ = tf.constant(background_volume,dtype=tf.float32)
            background_data_placeholder_ = background_reshaped
            background_mul = tf.multiply(background_data_placeholder_,
                                 background_volume_placeholder_)
            background_add = tf.add(background_mul, sliced_foreground)
            sliced_foreground = tf.clip_by_value(background_add, -1.0, 1.0)
        
        if True:
            stfts = tf.signal.stft(sliced_foreground, frame_length=model_settings['window_size_samples'], 
                                   frame_step=model_settings['window_stride_samples'], fft_length=None,
                                   window_fn=tf.signal.hann_window
                                   )
            spectrograms = tf.abs(stfts)
            num_spectrogram_bins = stfts.shape[-1]
            #default values used by contrib_audio.mfcc as shown here https://kite.com/python/docs/tensorflow.contrib.slim.rev_block_lib.contrib_framework_ops.audio_ops.mfcc
            lower_edge_hertz, upper_edge_hertz, num_mel_bins = 20.0, 4000.0, 40 
            linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix( num_mel_bins, num_spectrogram_bins, model_settings['sample_rate'],
                                                                                lower_edge_hertz, upper_edge_hertz)
            mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)
            mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))
            # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
            log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)
            # Compute MFCCs from log_mel_spectrograms and take the first 13.
            mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)[..., :model_settings['dct_coefficient_count']]
            mfccs = tf.reshape(mfccs,[model_settings['spectrogram_length'], model_settings['dct_coefficient_count'], 1])
            next_element['audio'] = mfccs
            next_element['label'] = tf.one_hot(next_element['label'],12)
        #next_element['audio'] = sliced_foreground
        return next_element
    
    return prepare_processing_graph

model_settings = prepare_model_settings(label_count, sample_rate, clip_duration_ms,
                               window_size_ms, window_stride_ms,
                               dct_coefficient_count,background_frequency)

ImportError: FloatProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
bg_path='/home/urmtha01/MountedLogs/BG_NOISE/'
BACKGROUND_NOISE_DIR_NAME='_background_noise_'
def prepare_background_data(bg_path,BACKGROUND_NOISE_DIR_NAME):
    """Searches a folder for background noise audio, and loads it into memory.
    It's expected that the background audio samples will be in a subdirectory
    named '_background_noise_' inside the 'data_dir' folder, as .wavs that match
    the sample rate of the training data, but can be much longer in duration.
    If the '_background_noise_' folder doesn't exist at all, this isn't an
    error, it's just taken to mean that no background noise augmentation should
    be used. If the folder does exist, but it's empty, that's treated as an
    error.
    Returns:
      List of raw PCM-encoded audio samples of background noise.
    Raises:
      Exception: If files aren't found in the folder.
    """
    background_data = []
    background_dir = os.path.join(bg_path, BACKGROUND_NOISE_DIR_NAME)
    if not os.path.exists(background_dir):
      return background_data
    #with tf.Session(graph=tf.Graph()) as sess:
    #    wav_filename_placeholder = tf.placeholder(tf.string, [])
    #    wav_loader = io_ops.read_file(wav_filename_placeholder)
    #    wav_decoder = contrib_audio.decode_wav(wav_loader, desired_channels=1)
    search_path = os.path.join(bg_path, BACKGROUND_NOISE_DIR_NAME,'*.wav')
    #for wav_path in gfile.Glob(search_path):
    #    wav_data = sess.run(wav_decoder, feed_dict={wav_filename_placeholder: wav_path}).audio.flatten()
    #    self.background_data.append(wav_data)
    for wav_path in gfile.Glob(search_path):
        #audio = tfio.audio.AudioIOTensor(wav_path)
        raw_audio = tf.io.read_file(wav_path)
        audio = tf.audio.decode_wav(raw_audio)
        background_data.append(audio[0])
    if not background_data:
        raise Exception('No background wav files were found in ' + search_path)
    return background_data
background_data = prepare_background_data(bg_path,BACKGROUND_NOISE_DIR_NAME)
#is_training=True
#background_volume_range_=0.1
#print(tf.reduce_max(background_data[5]))

In [ ]:
ds['train'] = ds['train'].map(get_preprocess_audio_func(model_settings,is_training=True,background_data=background_data))

In [ ]:
iterator = ds['train'].__iter__()

for i in range(5):
    next_element = iterator.get_next()
    tf.print(next_element['audio'].shape)